## 問題

https://twitter.com/e869120/status/1385725481920520193

## 解説

https://twitter.com/e869120/status/1386138990361726978

In [2]:
#r "nuget: FSharpPlus"

Installed Packages FSharpPlus, 1.3.2

In [3]:
open System
open System.Numerics
open System.Collections.Generic
open FSharpPlus

In [167]:
let [<Literal>] Mod = 1_000_000_007

In [168]:
let canPut height width (used: (int * int) -> bool) (x, y) =
  seq {
    for x' in x - 1 .. x + 1 do
      if 0 <= x' && x' < width then
        for y' in y - 1 .. y + 1 do
          // y' < height の条件を入れると動的計画法の最後の行で判定がおかしくなるので条件に含めない
          if 0 <= y' then yield used (x', y')
  } |> Seq.forall not
let canPutWithSet height width used (x, y) = canPut height width (fun xy -> used |> Set.contains xy) (x, y)
let canPutWithArray (used: bool[,]) (x, y) = canPut (Array2D.length1 used) (Array2D.length2 used) (fun (x, y) -> used[y, x]) (x, y)

In [177]:
let calcNextIndexes height width =
  // 置き方として実際にありえるパターンを洗い出し
  let dpIndexLists = Array.init width (fun _ -> ResizeArray<int>())
  let indexMaps = Array.init width (fun _ -> Dictionary<int, int * bool>())
  let rec loop position depth (dpIndex: int) used =
    let x, y = position % width, position / width
    let canPut = canPutWithSet height width used (x, y)
    if depth = width + 1 then
      let index = dpIndexLists[x].Count
      dpIndexLists[x].Add(dpIndex)
      indexMaps[x].Add(dpIndex, (index, canPut))
    else
      loop (position + 1) (depth + 1) dpIndex used
      if canPut then
        used |> Set.add (x, y) |> loop (position + 1) (depth + 1) (dpIndex + (1 <<< depth))
  for position = 0 to width - 1 do loop position 0 0 Set.empty
  let getIndexAndCanPut index key = indexMaps[index].TryGetValue(key) |> Option.ofPair |> Option.defaultValue (0, false)

  // 実際にありえる置き方に対応するインデックスを計算
  Array.init width (fun i ->
    dpIndexLists[i] |> Seq.map (fun dpIndex ->
      let dpIndexNoPut = dpIndex >>> 1
      let dpIndexPut = dpIndexNoPut + (1 <<< width)
      let index = (i + 1) % width
      let nextIndexNoPut = getIndexAndCanPut index dpIndexNoPut |> fst
      let nextIndexPut = if getIndexAndCanPut i dpIndex |> snd then getIndexAndCanPut index dpIndexPut |> fst |> Some else None
      nextIndexNoPut, nextIndexPut)
    |> Seq.toArray)

In [178]:
/// 動的計画法でパターンを求める
let calcDP (Cs: string[]) =
  let height = Cs.Length
  let width = Cs[0].Length
  let field = Array2D.init height width (fun i j -> Cs[i][j] = '.')
  let nextIndexArrays = calcNextIndexes height width
  // 列ごとにありえるパターン数は求めてあるのでその分だけの配列を確保
  let dp = Array2D.init (height + 1) width (fun _ j -> Array.zeroCreate<int> nextIndexArrays[j].Length)
  dp[0, 0][0] <- 1
  for i = 0 to height - 1 do
    for j = 0 to width - 1 do
      let ni, nj = if j + 1 = width then i + 1, 0 else i, j + 1
      nextIndexArrays[j] |> Array.iteri (fun k (nextIndexNoPut, nextIndexPut) ->
        if dp[i, j][k] <> 0 then
          dp[ni, nj][nextIndexNoPut] <- (dp[ni, nj][nextIndexNoPut] + dp[i, j][k]) % Mod
          match nextIndexPut, field[i, j] with
          | Some index, true -> dp[ni, nj][index] <- (dp[ni, nj][index] + dp[i, j][k]) % Mod
          | None, _ | _, false -> ())
  dp

In [179]:
let solve Cs =
  let Cs = Cs |> Seq.toArray
  let height = Cs.Length
  let dp = calcDP Cs
  (0, dp[height, 0]) ||> Seq.fold (fun result count -> (result + count) % Mod)

In [180]:
solve [
  "..."
]

5

In [181]:
solve [
  ".#."
  "#.."
  ".##"
]

13

In [182]:
solve [
  "######.##"
  "####..##."
  "..#...#.."
  "###...###"
  "#....##.#"
  ".##......"
  "#.####..#"
  "#.#######"
]

273768

In [183]:
solve [
  ".####...#.....#.#"
  ".#....#.#####...#"
  "#...##.##...#..##"
  "..#..####..#...##"
  ".#..#..#.#.##...#"
  ".#.#.#...#.##..#."
  "#...#..#..##..###"
  "###.#..###..###.."
  "...#.##.##.#....#"
  "..####....#.#...#"
  ".##...##.#.#...#."
  "..########...###."
  "#..##....#......."
  "##.##..###.#.##.."
  ".##....#........#"
  "....#####..##.#.."
  ".###...##..##.#.."
]

314465173

In [184]:
solve [
  ".##.##.#.#.#...##."
  "####.#..###.#.#..#"
  "#####.##...##.###."
  "...#.#.#.##.##.###"
  "..#.##.#.#....#..."
  "#.###.##....###..#"
  "....#####...#...#."
  ".#..##..#..###...."
  "....#..##.#.#..#.#"
  "###.#.....#..##.#."
  "#..#..#.#.##..###."
  "#...#....##..###.."
  "..#...#..###..##.."
  ".#....#.#.#..###.#"
  "##.#.#..#..###..##"
  "....###.##.##.##.."
  "#...####.#.#..##.."
  "..#.###.###.###.##"
  "#...##.#.#.#...#.#"
  "#..###..########.."
  "#.##.#####.#..#.##"
  "#..#........#...#."
]

47296634